<a href="https://colab.research.google.com/github/Cutie-tee/lab-chatbot-with-multi-query-retriever/blob/main/lab_chatbot_with_multi_query_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question Answering with LangChain, OpenAI, and MultiQuery Retriever

This interactive workbook demonstrates example of Elasticsearch's [MultiQuery Retriever](https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html) to generate similar queries for a given user input and apply all queries to retrieve a larger set of relevant documents from a vectorstore.

Before we begin, we first split the fictional workplace documents into passages with `langchain` and uses OpenAI to transform these passages into embeddings and then store these into Elasticsearch.

We will then ask a question, generate similar questions using langchain and OpenAI, retrieve relevant passages from the vector store, and use langchain and OpenAI again to provide a summary for the questions.

## Install packages and import modules

In [ ]:
!python3 -m pip install -qU jq lark langchain langchain-elasticsearch langchain_openai tiktoken

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai.llms import OpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from getpass import getpass

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.2/571.2 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.7/632.7 kB 11.8 MB/s eta 0:00:00


## Connect to Elasticsearch

ℹ️ We're using an Elastic Cloud deployment of Elasticsearch for this notebook. If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial.

We'll use the **Cloud ID** to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to https://cloud.elastic.co/deployments and select your deployment.

We will use [ElasticsearchStore](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html) to connect to our elastic cloud deployment, This would help create and index data easily.  We would also send list of documents that we created in the previous step

In [ ]:
 !pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.6 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata
os.environ['ELASTIC_API_KEY'] = userdata.get('ELASTIC_API_KEY')
import os
from google.colab import userdata
os.environ['ELASTIC_API_KEY'] = userdata.get('ELASTIC_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['ELASTIC_CLOUD_ID'] = userdata.get('ELASTIC_CLOUD_ID')

In [ ]:
from getpass import getpass
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import ElasticsearchStore
import os

def setup_vectorstore(index_name="indexlab001"):
    """
    Set up an Elasticsearch vector store with OpenAI embeddings.

    Args:
        index_name (str): Name of the Elasticsearch index to use

    Returns:
        ElasticsearchStore: Configured vector store instance
    """
    # Elastic Cloud configuration
    elastic_cloud_id = os.environ.get('ELASTIC_CLOUD_ID')

    # Securely get API keys
    elastic_api_key = getpass("Elastic Api Key:ELASTIC_API_KEY ")
    openai_api_key = getpass("OpenAI API key:OPENAI_API_KEY ")

    # Initialize OpenAI embeddings
    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

    # Create and return vector store
    return ElasticsearchStore(
        es_cloud_id=elastic_cloud_id,
        es_api_key=elastic_api_key,
        index_name=index_name,
        embedding=embeddings,
    )

if __name__ == "__main__":
    # Example usage
    vectorstore = setup_vectorstore()

Elastic Api Key:ELASTIC_API_KEY ··········
OpenAI API key:OPENAI_API_KEY ··········


## Indexing Data into Elasticsearch
Let's download the sample dataset and deserialize the document.

In [ ]:
from urllib.request import urlopen
import json

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/example-apps/chatbot-rag-app/data/data.json"

response = urlopen(url)
data = json.load(response)

with open("temp.json", "w") as json_file:
    json.dump(data, json_file)

### Split Documents into Passages

We’ll chunk documents into passages in order to improve the retrieval specificity and to ensure that we can provide multiple passages within the context window of the final question answering prompt.

Here we are chunking documents into 800 token passages with an overlap of 400 tokens.

Here we are using a simple splitter but Langchain offers more advanced splitters to reduce the chance of context being lost.

In [ ]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def metadata_func(record: dict, metadata: dict) -> dict:
    """
    Extract and populate metadata from JSON records.

    Args:
        record (dict): The JSON record being processed
        metadata (dict): The existing metadata dictionary

    Returns:
        dict: Updated metadata dictionary
    """
    # Extract metadata fields from the record if they exist
    metadata.update({
        'name': record.get('name', ''),
        'summary': record.get('summary', ''),
        'url': record.get('url', ''),
        'category': record.get('category', ''),
        'updated_at': record.get('updated_at', '')
    })

    return metadata

def load_and_split_documents(
    file_path: str,
    chunk_size: int = 1000,
    chunk_overlap: int = 200
) -> list:
    """
    Load and split JSON documents using LangChain.

    Args:
        file_path (str): Path to the JSON file
        chunk_size (int): Size of text chunks for splitting
        chunk_overlap (int): Number of characters to overlap between chunks

    Returns:
        list: List of processed document chunks
    """
    # Initialize the JSON loader
    loader = JSONLoader(
        file_path=file_path,
        jq_schema=".[]",  # Process each top-level array element
        content_key="content",
        metadata_func=metadata_func,
    )

    # Configure the text splitter
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        add_start_index=True  # Include chunk start position in metadata
    )

    # Load and split the documents
    docs = loader.load_and_split(text_splitter=text_splitter)

    return docs

if __name__ == "__main__":
    # Example usage
    file_path = "temp.json"
    documents = load_and_split_documents(file_path)

    # Print some information about the processed documents
    print(f"Processed {len(documents)} document chunks")
    if documents:
        print("\nSample document metadata:", documents[0].metadata)
        print("\nFirst chunk content preview:", documents[0].page_content[:200])

Processed 15 document chunks

Sample document metadata: {'source': '/content/temp.json', 'seq_num': 1, 'name': 'Work From Home Policy', 'summary': 'This policy outlines the guidelines for full-time remote work, including eligibility, equipment and resources, workspace requirements, communication expectations, performance expectations, time tracking and overtime, confidentiality and data security, health and well-being, and policy reviews and updates. Employees are encouraged to direct any questions or concerns', 'url': './sharepoint/Work from home policy.txt', 'category': 'teams', 'updated_at': '2020-03-01', 'start_index': 0}

First chunk content preview: Effective: March 2020
Purpose

The purpose of this full-time work-from-home policy is to provide guidelines and support for employees to conduct their work remotely, ensuring the continuity and produc


In [ ]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


def metadata_func(record: dict, metadata: dict) -> dict:
    #Populate the metadata dictionary with keys name, summary, url, category, and updated_at.
    None

    return metadata


# For more loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/
# And 3rd party loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/#third-party-loaders
loader = JSONLoader(
    file_path="temp.json",
    jq_schema=".[]",
    content_key="content",
    metadata_func=metadata_func,
)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=200  # define chunk size and chunk overlap
)

docs = loader.load_and_split(text_splitter=text_splitter)

### Bulk Import Passages

Now that we have split each document into the chunk size of 800, we will now index data to elasticsearch using [ElasticsearchStore.from_documents](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html#langchain.vectorstores.elasticsearch.ElasticsearchStore.from_documents).

We will use Cloud ID, Password and Index name values set in the `Create cloud deployment` step.

In [ ]:
import os
from google.colab import userdata
os.environ['ELASTIC_CLOUD_ID'] = userdata.get('ELASTIC_CLOUD_ID')
os.environ['ELASTIC_API_KEY'] = userdata.get('ELASTIC_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings

# ... other imports and code ...

# Get your environment variables
ELASTIC_CLOUD_ID = os.environ.get('ELASTIC_CLOUD_ID')
ELASTIC_API_KEY = os.environ.get('ELASTIC_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# Create vectorstore instance with Cloud ID and API key
vectorstore = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name="indexlab001",  # Provide a valid index name here
    embedding=embeddings,
)


llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(), llm)

In [ ]:
documents = vectorstore.from_documents(
    docs,
    embeddings,
    index_name="indexlab001",  # Provide a valid index name here
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
)

llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(), llm)
retriever

MultiQueryRetriever(retriever=VectorStoreRetriever(tags=['ElasticsearchStore', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.elasticsearch.ElasticsearchStore object at 0x7de4bf5ade70>, search_kwargs={}), llm_chain=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are an AI language model assistant. Your task is \n    to generate 3 different versions of the given user \n    question to retrieve relevant documents from a vector  database. \n    By generating multiple perspectives on the user question, \n    your goal is to help the user overcome some of the limitations \n    of distance-based similarity search. Provide these alternative \n    questions separated by newlines. Original question: {question}')
| OpenAI(client=<openai.resources.completions.Completions object at 0x7de4cc187b80>, async_client=<openai.resources.completions.AsyncCompletions object at 0x7de4cc1162c0>, temperature=0.0, model_kwargs={}, openai_api_k

# Question Answering with MultiQuery Retriever

Now that we have the passages stored in Elasticsearch, we can now ask a question to get the relevant passages.

In [ ]:
# ...other imports...
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


def metadata_func(record: dict, metadata: dict) -> dict:
    """
    Extract and populate metadata from JSON records.

    Args:
        record (dict): The JSON record being processed
        metadata (dict): The existing metadata dictionary

    Returns:
        dict: Updated metadata dictionary
    """
    # Extract metadata fields from the record if they exist
    metadata.update({
        'name': record.get('name', ''),
        'summary': record.get('summary', ''),
        'url': record.get('url', ''),
        'category': record.get('category', ''),
        'updated_at': record.get('updated_at', '')
    })

    return metadata

# For more loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/
# And 3rd party loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/#third-party-loaders
loader = JSONLoader(
    file_path="temp.json",
    jq_schema=".[]",
    content_key="content",
    metadata_func=metadata_func, # Pass the metadata function
)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=200  # define chunk size and chunk overlap
)

docs = loader.load_and_split(text_splitter=text_splitter) # Load and split using metadata


# ... Rest of the code...


In [ ]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import format_document

import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

LLM_CONTEXT_PROMPT = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Be as verbose and educational in your response as possible.

    context: {context}
    Question: "{question}"
    Answer:
    """
)

LLM_DOCUMENT_PROMPT = PromptTemplate.from_template(
    """
---
SOURCE: {name}
{page_content}
---
"""
)


def _combine_documents(
    docs, document_prompt=LLM_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)


_context = RunnableParallel(
    context=retriever | _combine_documents,
    question=RunnablePassthrough(),
)

chain = _context | LLM_CONTEXT_PROMPT | llm

ans = chain.invoke("what is the nasa sales team?")

print("---- Answer ----")
print(ans)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on the sales team at NASA?', '2. How does the sales team operate within NASA?', '3. What are the responsibilities of the NASA sales team?']


ValueError: Document prompt requires documents to have metadata variables: ['name']. Received document with missing metadata: ['name'].
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT 

**Generate at least two new iteratioins of the previous cells - Be creative.** Did you master Multi-
Query Retriever concepts through this lab?